In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
%matplotlib inline

In [3]:
os.chdir("/Users/Hatim/Documents/Semester 3/Machine Learning/kagalwala_hatim_assignment4")

In [4]:
credit_data = pd.read_csv("default.csv",index_col = 0)

In [5]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 1 to 30000
Data columns (total 24 columns):
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null int64
EDUCATION                     30000 non-null int64
MARRIAGE                      30000 non-null int64
AGE                           30000 non-null int64
PAY_0                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null int64
BILL_AMT2                     30000 non-null int64
BILL_AMT3                     30000 non-null int64
BILL_AMT4                     30000 non-null int64
BILL_AMT5                     30000 non-null int64
BILL_AMT6                     30000 non-null int64
PAY_AMT1                

In [6]:
cols_bill_amt = [col for col in credit_data.columns if ('BILL') in col]
cols_pay_amt = [col for col in credit_data.columns if ('PAY_AMT') in col]
cols_numerical = [col for col in credit_data.columns if ('AMT') in col]
cols_pay = [col for col in credit_data.columns if ('PAY_') in col]
cols_pay =cols_pay[0:6]

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, accuracy_score,f1_score,log_loss,precision_score,recall_score
from sklearn.feature_selection import RFECV
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVC
import warnings
from sklearn.ensemble import BaggingClassifier as BC
warnings.filterwarnings("ignore", category=DeprecationWarning)
scaler = StandardScaler()

#the columns from 1 to 10 are all categorical features. So we need to fit One Hot Encoder only on them.
categorical_features = [i for i in range(1,11)]
enc = OneHotEncoder(categorical_features=categorical_features)
y_target = credit_data["default payment next month"]

/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [8]:
#function to do all the data scaling and one hot encoding
def data_prep(data):
    #scaling the numerical values columns
    data[cols_numerical] = scaler.fit_transform(data[cols_numerical])
    #since the one hot encoder doesn't accept negative values, shifting each value by 2
    data[cols_pay] = data[cols_pay].apply(lambda x: x+2)
    #selection of all the rows for training
    x = data.iloc[:,0:23]
    #one hot encoding. Note that enc has been set on categorical features
    x = enc.fit_transform(x)
    return x

In [9]:
X_train = data_prep(credit_data)

In [10]:
#stratified sampling ensures that the test data follows the same pattern as the train data
X_train, X_test, y_train, y_test = train_test_split(X_train, y_target,test_size=0.2, 
                                                    stratify=credit_data["default payment next month"], 
                                                    random_state=42)

In [9]:
def gridsearch(X_train,y_train,X_test,y_test,model, params, scoring,cv=None):
    gs_cv = GridSearchCV(model,params,scoring=scoring,n_jobs=-1,cv=cv)
    gs_cv.fit(X_train, y_train)
    y_pred_train = gs_cv.predict(X_train)
    y_pred_test = gs_cv.predict(X_test)
    print("----------------------------TRAINING DATA----------------------------")
    print("Accuracy on training set is: "+str(accuracy_score(y_train,y_pred_train)))
    print("F1 score on training set is: "+str(f1_score(y_train,y_pred_train)))
    print("Precision on training set is: "+str(precision_score(y_train,y_pred_train)))
    print("Recall on training set is: "+str(recall_score(y_train,y_pred_train)))
    print("----------------------------TEST DATA----------------------------")     
    print("Accuracy on test set is: "+str(accuracy_score(y_test,y_pred_test)))
    print("F1 score on test set is: "+str(f1_score(y_test,y_pred_test)))
    print("Precision on test set is: "+str(precision_score(y_test,y_pred_test)))
    print("Recall on test set is: "+str(recall_score(y_test,y_pred_test)))
    print("-----------------------------------------------------------------")
    print("BEST PARAMETERS BASED ON "+scoring+" AS A SCORING PARAMETER")  
    print("Best parameters: "+str(gs_cv.best_params_))

In [10]:
svm_parameters = {'C': [0.001, 0.01, 0.1, 1, 10],'gamma':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}

In [11]:
svm = SVC(kernel="rbf",tol=0.01,cache_size=1000)

In [16]:
gridsearch(X_train,y_train,X_test,y_test,svm,svm_parameters,"accuracy",3)

----------------------------TRAINING DATA----------------------------
Accuracy on training set is: 0.855083333333
F1 score on training set is: 0.571040947213
Precision on training set is: 0.827081100393
Recall on training set is: 0.436051987192
----------------------------TEST DATA----------------------------
Accuracy on test set is: 0.807333333333
F1 score on test set is: 0.39029535865
Precision on test set is: 0.650263620387
Recall on test set is: 0.278824415976
-----------------------------------------------------------------
BEST PARAMETERS BASED ON accuracy AS A SCORING PARAMETER
Best parameters: {'C': 1, 'gamma': 0.1}


In [ ]:
svm = SVC(kernel="rbf",tol=0.01,cache_size=1000,class_weight={1:0.9,0:0.1})
svm_parameters = {'C': [0.001, 0.01, 0.1, 1, 10],'gamma':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
gridsearch(X_train,y_train,X_test,y_test,svm,svm_parameters,"",3)

In [19]:
import dask_searchcv as dcv

In [ ]:
svm_parameters = {'C': [0.001, 0.01, 0.1, 1, 10],'gamma':[0.001, 0.01, 0.1, 1, 10, 100, 1000],
                  'class_weight': ['balanced',None,{1:0.8,0:0.2},{1:0.9,0:0.1},{1:0.7,0:0.3},{1:0.6,0:0.4}]}
svm = SVC(kernel="rbf",tol=0.01,cache_size=1000)
search = dcv.GridSearchCV(svm, svm_parameters, cv=3)
search.fit(X_train,y_train)

In [123]:
#here I try to use bagging.

n_estimators = 400
clf = BC(base_estimator=SVC(kernel="rbf",C=10,gamma=10,cache_size=1000),
         max_samples=1/n_estimators, n_estimators=n_estimators,n_jobs=-1)
clf.fit(X_train,y_train)

BaggingClassifier(base_estimator=SVC(C=10, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=10, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.0025, n_estimators=400, n_jobs=-1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [124]:
y_pred_train = clf.predict(X_train)

In [125]:
y_pred_test = clf.predict(X_test)

In [126]:
print("----------------------------TRAINING DATA----------------------------")
print("Accuracy on training set is: "+str(accuracy_score(y_train,y_pred_train)))
print("F1 score on training set is: "+str(f1_score(y_train,y_pred_train)))
print("Precision on training set is: "+str(precision_score(y_train,y_pred_train)))
print("Recall on training set is: "+str(recall_score(y_train,y_pred_train)))
print("----------------------------TEST DATA----------------------------")     
print("Accuracy on test set is: "+str(accuracy_score(y_test,y_pred_test)))
print("F1 score on test set is: "+str(f1_score(y_test,y_pred_test)))
print("Precision on test set is: "+str(precision_score(y_test,y_pred_test)))
print("Recall on test set is: "+str(recall_score(y_test,y_pred_test)))

----------------------------TRAINING DATA----------------------------
Accuracy on training set is: 0.778791666667
F1 score on training set is: 0.0
Precision on training set is: 0.0
Recall on training set is: 0.0
----------------------------TEST DATA----------------------------
Accuracy on test set is: 0.778833333333
F1 score on test set is: 0.0
Precision on test set is: 0.0
Recall on test set is: 0.0


/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [131]:
param_grid={"n_estimators":[10,50,100,500,1000,2000],'max_samples': [0.1,0.5,0.6, 0.8, 1.0]}

In [20]:
#trying downsampling
X=credit_data.sample(10000)
y=X["default payment next month"]

In [21]:
X_train=data_prep(X)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y,test_size=0.2, 
                                                    stratify=X["default payment next month"], 
                                                    random_state=42)

In [24]:
gridsearch(X_train,y_train,X_test,y_test,svm,svm_parameters,"accuracy",3)

----------------------------TRAINING DATA----------------------------
Accuracy on training set is: 0.8545
F1 score on training set is: 0.574250182882
Precision on training set is: 0.810113519092
Recall on training set is: 0.444759206799
----------------------------TEST DATA----------------------------
Accuracy on test set is: 0.8055
F1 score on test set is: 0.40971168437
Precision on test set is: 0.619266055046
Recall on test set is: 0.30612244898
-----------------------------------------------------------------
BEST PARAMETERS BASED ON accuracy AS A SCORING PARAMETER
Best parameters: {'C': 10, 'gamma': 0.01}


In [25]:
gridsearch(X_train,y_train,X_test,y_test,svm,svm_parameters,"roc_auc",3)

----------------------------TRAINING DATA----------------------------
Accuracy on training set is: 0.8025
F1 score on training set is: 0.257518796992
Precision on training set is: 0.754820936639
Recall on training set is: 0.155240793201
----------------------------TEST DATA----------------------------
Accuracy on test set is: 0.7895
F1 score on test set is: 0.188824662813
Precision on test set is: 0.628205128205
Recall on test set is: 0.111111111111
-----------------------------------------------------------------
BEST PARAMETERS BASED ON roc_auc AS A SCORING PARAMETER
Best parameters: {'C': 10, 'gamma': 0.001}


In [26]:
gridsearch(X_train,y_train,X_test,y_test,svm,svm_parameters,"f1",3)

/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metric

/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Hatim/anaconda/lib/python3.6/site-packages/sklearn/metric

----------------------------TRAINING DATA----------------------------
Accuracy on training set is: 0.8545
F1 score on training set is: 0.574250182882
Precision on training set is: 0.810113519092
Recall on training set is: 0.444759206799
----------------------------TEST DATA----------------------------
Accuracy on test set is: 0.8055
F1 score on test set is: 0.40971168437
Precision on test set is: 0.619266055046
Recall on test set is: 0.30612244898
-----------------------------------------------------------------
BEST PARAMETERS BASED ON f1 AS A SCORING PARAMETER
Best parameters: {'C': 10, 'gamma': 0.01}


In [29]:
def fitting_svm(lr_):
    print("Accuracy score on train data: "+str(lr_.score(X_train,y_train)))
    print("Accuracy score on the test data: "+str(lr_.score(X_test,y_test)))
    y_pred_default = lr_.predict(X_test)
    print("F1 score on the test data: "+str(f1_score(y_test,y_pred_default)))

    print("The log loss is: "+ str(log_loss(y_test,(lr_.predict_proba(X_test)))))

In [31]:
svm = SVC(C=10,gamma =0.01, kernel="rbf",tol=0.01,cache_size=1000,probability=True)
svm.fit(X_train,y_train)
fitting_svm(svm)

Accuracy score on train data: 0.8545
Accuracy score on the test data: 0.8055
F1 score on the test data: 0.40971168437
The log loss is: 0.47436218095
